In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
from tqdm import tqdm

In [40]:
#ukraine
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?title=Ukraine Russia&begin_date=2022-02-24&end_date=2022-06-23&orderByTime=yes&page_no={}'
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Ukraine&begin_date=2022-02-24&end_date=2022-08-23&orderByTime=yes&page_no={}'
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Ukraine&begin_date=2021-08-24&end_date=2022-02-23&orderByTime=yes&page_no={}'

#Gaza
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Gaza Israel Palestine&begin_date=2023-10-06&end_date=2024-03-07&orderByTime=yes&page_no={}'
gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Gaza Israel Palestine&begin_date=2023-03-0&end_date=2023-10-06&orderByTime=yes&page_no={}'

In [41]:
links = []
for page_number in tqdm(range(1, 50)):  #loop from page 1 to 165
    gt_url = gt_base_url.format(page_number)
    #print(gt_url)
    #print("Processing page:", page_number)
    
    r = requests.get(gt_url)
    b = BeautifulSoup(r.content, 'html.parser')

    for news in b.findAll('div', {'class': 'span9'}):
        link = news.find('a')['href']
        links.append(link)

print("Total links collected:", len(links))
print("Sample links:", links[:5])

100%|██████████| 49/49 [00:35<00:00,  1.38it/s]

Total links collected: 482
Sample links: ['https://www.globaltimes.cn/page/202309/1299106.shtml', 'https://www.globaltimes.cn/page/202309/1299020.shtml', 'https://www.globaltimes.cn/page/202309/1298931.shtml', 'https://www.globaltimes.cn/page/202309/1298698.shtml', 'https://www.globaltimes.cn/page/202309/1298601.shtml']


In [42]:
#initialize webdriver
driver = webdriver.Chrome()

article_titles = []
authors = []
published_times = []
article_texts = []
article_categories_one = []
article_categories_two = []
picture_descriptions = []
keywords = []
descriptions = []
authors_description_links = []

#loop through each link
for link in tqdm(links):
    #navigate to the page containing the article
    driver.get(link)

    #extract the HTML content of the page
    html_content = driver.page_source

    #parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    #find article title
    article_title_elem = soup.find("div", class_="article_title")
    article_title = article_title_elem.text.strip() if article_title_elem else "Title not found"
    article_titles.append(article_title)

    #find author and published time
    author_published_time = soup.find("div", class_="author_share_left")
    if author_published_time == None:
        print("Author and published time not found, trying again")
        driver.get(link)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        author_published_time = soup.find("div", class_="author_share_left")

    published_time = author_published_time.find("span", class_="pub_time").text.strip() if author_published_time else "Published time not found"
    published_times.append(published_time)

    ##find author
    author_elem = soup.find("span", class_="byline")

    iter = 0
    while author_elem == None:
        print("Author not found, trying again")
        driver.get(link)
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        author_elem = soup.find("span", class_="byline")
        iter += 1
        if iter > 5: break
    author = author_elem.text.strip() if author_elem else "Author not found"
    author_card = author_elem.find("div", class_ = "card_author_name") if author_elem else None

    if author_card:
        author_description_link = author_card.find("a")["href"]
        authors_description_links.append(author_description_link)
        author = author_card.find("a").text
    authors.append(author)

    #find article text
    article_content_elem = soup.find("div", class_="article_content")
    article_text = article_content_elem.text.strip() if article_content_elem else "Article text not found"
    article_texts.append(article_text)

    #articla category
    article_column_elem = soup.find("div", class_="article_column")
    if article_column_elem:
        article_category_one_elem = article_column_elem.find("span", class_= "cloumm_level_one")
        aticlal_category_one = article_category_one_elem.text if article_category_one_elem else "Category one not found"
        article_categories_one.append(aticlal_category_one)

        article_category_two_elem = article_column_elem.find("span", class_= "cloumm_level_two")
        article_category_two = article_category_two_elem.text if article_category_two_elem else "Category two not found"
        article_categories_two.append(article_category_two)
    else:
        article_categories_one.append("Category one not found")
        article_categories_two.append("Category two not found")

    #picture description
    picture_description_elem = soup.find("p", class_="picture")
    picture_description = picture_description_elem.text.strip() if picture_description_elem else "Picture description not found"
    picture_descriptions.append(picture_description)

    #keywords
    meta_keywords_elem = soup.find("meta", id= "MetaKeywords")["content"] if soup.find("meta", id= "MetaKeywords") else None
    meta_keywords =  meta_keywords_elem.strip() if meta_keywords_elem else "Keywords not found"
    keywords.append(meta_keywords)

    #description
    meta_description_elem = soup.find("meta", id= "MetaDescription")["content"] if soup.find("meta", id= "MetaKeywords") else None
    meta_description =  meta_description_elem.strip() if meta_description_elem else "Description not found"
    descriptions.append(meta_description)

#close webdriver
driver.quit()

 12%|█▏        | 60/482 [01:49<10:28,  1.49s/it]

Author and published time not found, trying again


 23%|██▎       | 112/482 [03:18<10:20,  1.68s/it]

Author not found, trying again
Author not found, trying again
Author not found, trying again
Author not found, trying again
Author not found, trying again
Author not found, trying again


100%|██████████| 482/482 [12:12<00:00,  1.52s/it]


In [43]:
#authors details

driver = webdriver.Chrome()

authors_names = []
authors_titles = []
authors_descriptions = []

for link in authors_description_links:
    #navigate to the page containing the article
    driver.get(link) if link.startswith("http") else driver.get("https://www.globaltimes.cn" + link)

    #extract the HTML content of the page
    html_content = driver.page_source

    #parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    #find author name
    author_name_elem = soup.find("div", class_="author_name")
    author_name = author_name_elem.text.strip() if author_name_elem else "Author name not found"
    authors_names.append(author_name)

    #find author title
    author_title_elem = soup.find("div", class_="author_title")
    author_title = author_title_elem.text.strip() if author_title_elem else "Author title not found"
    authors_titles.append(author_title)

    #find author description
    author_description_elem = soup.find("div", class_="author_desc")
    author_description = author_description_elem.text.strip() if author_description_elem else "Author description not found"
    authors_descriptions.append(author_description)

#close webdriver
driver.quit()

In [44]:
#save to csv file
data = {
    "article_title": article_titles,
    "author": authors,
    "published_time": published_times,
    "article_text": article_texts,
    "article_category_one": article_categories_one,
    "article_category_two": article_categories_two,
    "picture_description": picture_descriptions
}

df = pd.DataFrame(data)
df.to_csv("Gaza_textcontain_before_new.csv", index=False)

In [36]:
df

,article_title,author,published_time,article_text,article_category_one,article_category_two,picture_description
0,Palestinian death toll in Gaza from Israeli at...,By Xinhua,"Published: Mar 07, 2024 10:34 AM",A young man walks past destroyed buildings in ...,PHOTO,WORLD,A young man walks past destroyed buildings in ...
1,Daily life at Al-Maghazi refugee camp in Gaza ...,By Xinhua,"Published: Mar 07, 2024 11:44 AM",A vendor makes desserts at Al-Maghazi refugee ...,PHOTO,WORLD,A vendor makes desserts at Al-Maghazi refugee ...
2,Two-State solution only way to break vicious c...,By Global Times,"Published: Mar 07, 2024 12:50 PM",People walk past destroyed buildings in the so...,CHINA,POLITICS,People walk past destroyed buildings in the so...
3,Biden to deliver State of the Union address,By GT staff reporters,"Published: Mar 07, 2024 10:28 PM",US President Joe Biden speaks during a meeting...,CHINA,DIPLOMACY,US President Joe Biden speaks during a meeting...
4,Houthi attacks in Red Sea reports first fatali...,By Fan Anqi,"Published: Mar 07, 2024 10:58 PM",A container ship sails in the Red Sea in the S...,CHINA,DIPLOMACY,A container ship sails in the Red Sea in the S...
...,...,...,...,...,...,...,...
485,Political efforts must be intensified to solve...,By Xie Wenting and Bai Yunyi,"Published: Oct 22, 2023 06:32 PM",Teams conduct a search and rescue operation un...,IN-DEPTH,DIPLOMATIC CHANNEL,Teams conduct a search and rescue operation un...
486,US biased attitude in Israel-Palestine conflic...,By Ma Xiaolin,"Published: Oct 22, 2023 08:21 PM",Illustration: Liu Rui/ GTAny act that leads to...,OPINION,VIEWPOINT,Illustration: Liu Rui/ GT
487,US inability to reconcile Israel-Palestine con...,By Global Times,"Published: Oct 22, 2023 08:32 PM","NUSEIRAT, GAZA - OCTOBER 22: A view of destruc...",OPINION,OBSERVER,"NUSEIRAT, GAZA - OCTOBER 22: A view of destruc..."
488,Israel’s deepening attacks in Gaza likely to e...,Zhao Yusha,"Published: Oct 22, 2023 09:31 PM Updated: Oct ...",Leaders and top officials from more than a doz...,CHINA,DIPLOMACY,Leaders and top officials from more than a doz...


In [45]:
#authors details
authors_data = {
    "author_name": authors_names,
    "author_title": authors_titles,
    "author_description": authors_descriptions
}

authors_data = pd.DataFrame(authors_data)
authors_data.to_csv("Gaza_textcontain_before_new_authors.csv", index=False)
authors_data

,author_name,author_title,author_description
0,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
1,Yang Sheng,"Reporter, Beijing",Yang Sheng is a chief reporter at the Global T...
2,Deng Xiaoci,"Reporter, Beijing",Global Times reporter tracking China's space p...
3,Xu Liuliu,"Reporter, Beijing","Global Times senior reporter, mainly covers th..."
4,Deng Xiaoci,"Reporter, Beijing",Global Times reporter tracking China's space p...
5,Yang Sheng,"Reporter, Beijing",Yang Sheng is a chief reporter at the Global T...
6,Liu Xin,"Reporter, Beijing",Liu Xin co-leads the Global Times China desk. ...
7,Wang Qi,"Reporter, Beijing",Global Times reporter covering international a...
8,Bradley Blankenship,Author title not found,Bradley Blankenship is a Prague-based American...
9,Yang Sheng,"Reporter, Beijing",Yang Sheng is a chief reporter at the Global T...
